In [1]:
using MATLAB
using Ipopt
using JuMP

In [174]:
mat"""
sys_case=39;
compute_gain;
$gain_mtx=gain_mtx;
$w_0=E*delta0;
$num_line=num_line;
$num_bus=num_bus;
"""
size_w=convert(Int64,num_line);
size_u=convert(Int64,num_bus);

>> >> >> >> Progress: 1/4
Progress: 2/4
Progress: 3/4
Progress: 4/4


In [175]:
c=ones(size_u);
c=zeros(size_u); c[1]=1;
D=diagm(sign.(w_0))

m = Model(solver=IpoptSolver());
@variable(m, 0 <= w[1:size_w] <= pi/2,start=0)
@variable(m, 0 <= sinw[1:size_w] <= 1,start=0)
@variable(m, 0 <= u[1:size_u] <= pi/2,start=0)

@NLconstraint(m, myconstr[i=1:size_w], sinw[i] <= sin(abs(w_0[i])+w[i]))
@constraint(m,gain_mtx[3]*u.<=(eye(size_w)-gain_mtx[4]*diagm(cos.(w_0)))*w-gain_mtx[4]*D*sin.(w_0)+gain_mtx[4]*sinw)
@objective(m, Max, c'*u)
solve(m)
println("w = ", getvalue(w), " u = ", getvalue(u))

sgt_mtx=eye(size_w)-gain_mtx[4]*(diagm(cos.(w_0).*getvalue(w)+D*sin.(w_0)-sin.(abs.(w_0)+getvalue(w))));
eigmin(sgt_mtx)

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6072
Number of nonzeros in Lagrangian Hessian.............:       46

Total number of variables............................:      130
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      130
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       92
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       92

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

0.9653236783697489

In [185]:
w_max=pi/12*ones(size_w);
sgt_mtx=eye(size_w)-gain_mtx[4]*(diagm(cos.(w_0).*w_max+D*sin.(w_0)-sin.(abs.(w_0)+w_max)));
eigmin(sgt_mtx)
#maximum(getvalue(w))
pi/12

0.2617993877991494